In [1]:
import pandas as pd
import numpy as np

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed')).History will not be written to the database.


In [11]:
def count(categorical_var, count_var, dataframe):
    counts_df = dataframe.value_counts([categorical_var, count_var])
    counts_df = counts_df.groupby(categorical_var).sum()
    counts_df = counts_df.sort_values(ascending=False)
    counts_df = pd.DataFrame(counts_df.reset_index().values,
                             columns=[categorical_var, count_var])
    return counts_df


def count_unique(categorical_var, count_var, dataframe, cummul=False, unique=False):
    counts_df = count(categorical_var, count_var, dataframe)
    new_df = []
    terms = []
    for prog in counts_df[categorical_var].unique():
        terms_ = dataframe.loc[dataframe[categorical_var] == prog, count_var].unique()
        unique_terms = [term for term in terms_ if term not in terms]
        terms.extend(unique_terms)
        new_df.append([prog, len(unique_terms)])
    new_df = pd.DataFrame(new_df, columns=[categorical_var, count_var])
    if cummul:
        new_df[count_var] = new_df[count_var].cumsum()
    if unique:
        return new_df
    else:
        return counts_df

# Data

| program | geneset | p-value | adjusted p-value |
|---------|---------|---------|------------------|
| program1 | genesetA | 0.01 | 0.01 |
| program1 | genesetA | 0.02 | 0.02 |
| program2 | genesetA | 0.03 | 0.03 |
| program2 | genesetA | 0.04 | 0.04 |
| program3 | genesetB | 0.05 | 0.05 |


In [16]:
# Now take the above markdown and convert it to code
data = pd.DataFrame({
    'program': ['program1', 'program1', 'program2', 'program2', 'program3'],
    'geneset': ['genesetA', 'genesetA', 'genesetA', 'genesetA', 'genesetB'],
    'p-value': [0.01, 0.02, 0.03, 0.04, 0.05],
    'adjusted p-value': [0.01, 0.02, 0.03, 0.04, 0.05]
})
sig_var = 'adjusted p-value'
count_var = 'geneset'
categorical_var = 'program'
data

,program,geneset,p-value,adjusted p-value
0,program1,genesetA,0.01,0.01
1,program1,genesetA,0.02,0.02
2,program2,genesetA,0.03,0.03
3,program2,genesetA,0.04,0.04
4,program3,genesetB,0.05,0.05


# 1. All enriched terms
Count everything, including terms enriched multiple times in the same program and terms enriched in multiple programs.

```python
count(categorical_var=categorical_var, count_var=count_var, dataframe=unique_data)
```
| program | geneset |
|---------|---------|
| program1 | 2 |
| program2 | 2 |
| program3 | 1 |

In [13]:
count(categorical_var, count_var, data)

,program,geneset
0,program1,2
1,program2,2
2,program3,1


# 2. Unique within a program, but can be repeated across programs. 
i.e if `program1` and `program2` are both enriched for `genesetA` we count it for both programs
    
```python
unique_data = data.drop_duplicates(subset=[categorical_var, count_var])
count(categorical_var=categorical_var, count_var=count_var, dataframe=unique_data)
```
| program | geneset |
|---------|---------|
| program1 | 1 |
| program2 | 1 |
| program3 | 1 |

In [19]:
unique_data = data.drop_duplicates(subset=[categorical_var, count_var])
count(categorical_var=categorical_var, count_var=count_var, dataframe=unique_data)

,program,geneset
0,program1,1
1,program2,1
2,program3,1


# 3. Unique across all programs. 
i.e. if `program1` and `program2` are both enriched for `genesetA`, but `program1` has a much lower adjusted p-value, we only count `genesetA` for program1

```python
unique_data = data.sort_values(by=sig_var)
unique_data = unique_data.drop_duplicates(subset=count_var)
unique_df = count_unique(categorical_var=categorical_var, count_var=count_var, dataframe=unique_data)
unique_df = unique_df.sort_values(count_var, ascending=False)
```

| program | geneset |
|---------|---------|
| program1 | 1 |
| program3 | 1 |

In [21]:
unique_data = data.drop_duplicates()
unique_data = data.sort_values(by=sig_var)
unique_data = unique_data.drop_duplicates(subset=count_var)
unique_df = count_unique(categorical_var=categorical_var, count_var=count_var, dataframe=unique_data)
unique_df = unique_df.sort_values(count_var, ascending=False)
unique_df

,program,geneset
0,program1,1
1,program3,1


# Unique 